# SESAMME Demo

## 1) Goals and introduction <a name="Introduction"></a>

This notebook will walk users through the functionality of **SESAMME - 
Simultaneous Estimates of Star-cluster Age, Metallicity, Mass, and Extinction -** v1.0, which is a Python script for Bayesian inference of the properties of extragalactic star clusters.

SESAMME is built on a combination of the affine invariant Markov chain Monte Carlo algorithm `emcee` (Foreman-Mackey et al. 2013) and user-specified suites of stellar population models (SSPs - for example, BPASS v2.2 "Tuatara" (Stanway & Eldridge 2018) and/or v2.3 "Broc" (Bryne+ 2022)). As the name implies, SESAMME v1.0 marginalizes over four physical parameters - the age, metallicity, and sometimes the mass of the star cluster, as well as the degree of reddening along the line of sight.

By the end of this tutorial, you will have a working example of how to:
1. Set up and execute a SESAMME run; and
2. Perform standard post-processing analyses of the resulting fits.


For an example of how to prepare a SESAMME model cube, please see the accompanying notebook SESAMME Cubemaker Demo. That notebook also shows an example of how to smooth and rebin a spectrum for comparison with BPASS SSP models, with the caveat that our methods may not be optimal for your science uses. 

1. [Introduction](#Introduction)
2. [Imports](#Imports)
3. [Setting up SESAMME's meta-parameters](#Setup)
4. [Starting a SESAMME run](#Run)

5. [Post-run analyses](#Post)
    1. [Cleaning the MCMC chain](#Burn)
    2. [Corner plots and statistics](#Corner)
    3. [Visualizing a likely model](#Visualization)

## 2) Imports <a name="Imports"></a>

This notebook (and SESAMME generally) uses the following packages:

* **numpy** and **scipy** for handling array functions
* **astropy** for handling FITS files and the astropy-affiliated package **specutils** for manipulating spectra
* **emcee** (https://emcee.readthedocs.io/en/stable/) for the MCMC machinery
* **extinction** (https://extinction.readthedocs.io/en/latest/) and **dust_extinction** (https://dust-extinction.readthedocs.io/en/stable/) for accessing extinction curves
* **matplotlib** and **corner** for data visualization 

If you do not have these packages installed, you can install them using pip or conda.

In [ ]:
### File and unit handling tools
from astropy.io import fits
import astropy.units as u
from astropy import constants as const


### Dust models
import extinction, dust_extinction
from dust_extinction.parameter_averages import G23
from dust_extinction.averages import G03_LMCAvg, G03_SMCBar


### Manipulating arrays
import numpy as np
from scipy import interpolate
from specutils import Spectrum1D
import emcee


### Data visualization
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, SymLogNorm, ListedColormap, BoundaryNorm, Normalize
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import corner
from IPython.display import display, Math

plt.rcParams.update({'font.size': 12, 'font.weight': 'semibold', 'axes.labelsize': 12, 
                     'axes.labelweight':'semibold'})

%matplotlib notebook


import os, sys, warnings, time

os.environ["OMP_NUM_THREADS"] = "1"

## 3) Setting up SESAMME's meta-parameters

Before using SESAMME to model your spectrum, there are several meta-parameters that should be tweaked to fit your use case. These include specifying the attenuation curve used in the modeling; the size of the walker ensemble; the "ball" of initial positions for the walker ensemble; and ranges on the priors for each of the four variables.


**Choosing an attenuation curve**

During the model process, SESAMME will redden a model SSP by a specified amount using one of several attenuation curves. See the documentation for reference information on each curve. The options include:
* Five options for a Milky Way-like curve ('CCM', 'ODonnell', 'Fitzpatrick99', 'FitzMassa07', 'Gordon23')
* Curves based on the Large Magellanic Cloud ('LMC') and Small Magellanic Cloud ('SMC')
* A starbursting galaxy curve ('Calzetti')

The default option in the script version of SESAMME is the Milky Way attenuation curve of Cardelli, Clayton, & Mathis (1989). 

In [ ]:
# Set the attenuation curve
use_red_law = 'CCM'

**Creating and initializing the emcee walker ensemble**

emcee, the statistical backbone of SESAMME, uses an ensemble of "walkers" to explore the parameter space and determine what values provide likely fits to your data. You must specify the number of walkers prior to running SESAMME, as well as the number of steps in the chain. In general, more walkers iterating for fewer steps gives better constraints on the properties of a star cluster than few walkers iterating for many steps. The default number of walkers is 128. 

You must also initialize the walkers in parameter space. Here, we do so using normal distributions centered on arbitrary values of log(age), log(Z), E(B-V), and amplitude log(A), each with a relatively narrow dispersion. The default values are reasonable approximations for a young, massive, and sub-solar metallicity star cluster.

In [ ]:
# Set the number and dimensionality of the emcee walker ensemble
# ndim MUST be set to 4
nwalkers, ndim = 128, 4
nsteps = 10000


# Set the initial positions of the walker ensemble
initial_pos = [7., -2.0, 0.2, -2.0] + ([0.1, 0.1, 0.1, 0.1] * np.random.randn(nwalkers, ndim))

**Establish the parameter space and constrict the priors**

In order to generate models during the MCMC process, SESAMME needs to know what its options are for values of metallicity and age. These need to be explicitly set by the user (as a dictionary) according to the SSP model set being used. We show an example of the required formatting below, using the BPASS metallicity and age ranges.

Given these intervals, the parameter space explore by the MCMC walker ensemble in a given run can be further restricted through setting priors. By default, priors for all four variables have relatively broad ranges, but can be further restricted according to expectations about the age of the star cluster to be modeled, its metallicity, etc.

Priors on the age, metallicity, and rescaling parameter A should be given in log-units, while E(B-V) is linear. E.g., if you're confident that your cluster is between 1 and 10 Myr old, you should respectively set 'age_low' and 'age_hi' to 6.0 and 7.0 in the dictionary defined below.

In [ ]:
# Dictionary defining both the labels and values of the predefined BPASS model range in metallicity Z
metal_dict = {'Z001' : 0.001,
              'Z002' : 0.002,
              'Z003' : 0.003,
              'Z004' : 0.004,
              'Z006' : 0.006,
              'Z008' : 0.008,
              'Z010' : 0.010,
              'Z014' : 0.014,
              'Z020' : 0.020,
              'Z030' : 0.030,
              'Z040' : 0.040,
              'Zem4' : 1e-4,
              'Zem5' : 1e-5
}

# Dictionary defining both the labels and values of the predefined BPASS model range in age
age_dict = {}
for n in range(2, 53):
    # For BPASS, this yields ages in steps of 0.1 dex beginning at log(age) = 6.0 and ending at log(age) = 11.0
    t = round(np.log10(10**(6+0.1*(n-2))), 1)
    age_dict[str(t)] = t

In [ ]:
# The upper and lower bounds for priors on each variable
prior_ranges = {
    'age': [6.0, 7.0],
    'met': [-3.0, -1.5],
    'ebv': [0.01, 1.0],
    'amp': [-20.0, 1.0]
}

In [ ]:
def check_prior_ranges(priors):
    
    # Ensure that the first value in the prior-boundary pair is lower than the second
    for key, value in priors.items():
        if value[0] > value[1]:
            raise ValueError("Prior boundaries are out of order for variable "+"\'"+key+"\'")
            
    # Check that the priors make physical sense
    for key in ['age', 'met', 'ebv']:
        if (key == 'age') & ((priors[key][0] < 5.) | (priors[key][1] < 5.) | (priors[key][0] > 11.) | (priors[key][1] > 11.)):
            warnings.warn("\n\n Your age prior may extend to unphysically young and/or old values. Double check before proceeding with SESAMME.")
        if (key == 'met') & ((priors[key][0] < -5) | (priors[key][1] < -5) | (priors[key][0] > -1.3) | (priors[key][1] > -1.3)):
            warnings.warn("\n\n Your metallicity prior may extend to unphysical values. Double check before proceeding with SESAMME.")
        if (key == 'ebv') & ((priors[key][0] < 0) | (priors[key][1] < 0) | (priors[key][0] > 100) | (priors[key][1] > 100)):
            warnings.warn("\n\n Your E(B-V) prior may extend to unphysical values. Double check before proceeding with SESAMME.")

The functions below (up until the start of 4. [Starting a SESAMME run](#Run)) are either key inputs to `emcee` for the MCMC process, or are there to parse the input model cube in a way that is compatible with other user inputs (e.g., the reddening and inclusion of nebular continuum). In general these should not be edited in any way.

In [ ]:
def log_prior(theta):
    """
    Calculate a flat prior probability within the bounds prescriped below.
    
    Inputs:
    - A 4-tuple or 4-element list/array theta containing, in order, a log(age) logt, 
    a log(metallicity) logZ, an E(B-V) value ebv, and an amplitude ampl.
    Change the limits of each parameter to fit your use case.
    
    Output:
    - 0 if parameters are within allowed ranges, or -inf otherwise
    """
    
    # Ensure the priors make sense...
#     global prior_ranges
    check_prior_ranges(prior_ranges)
    
    # ...Then use them to set the flat prior probability
    
    logt, logZ, ebv, ampl = theta
    
    if prior_ranges['age'][0] <= logt <= prior_ranges['age'][1] and \
    prior_ranges['met'][0] <= logZ <= prior_ranges['met'][1] and \
    prior_ranges['ebv'][0] < ebv < prior_ranges['ebv'][1] and \
    prior_ranges['amp'][0] < ampl < prior_ranges['amp'][1] :
        return 0.0
    else:
        return -np.inf

def log_likelihood(y, yerr, y_model, mask):
    """
    Evaluate the likelihood function by comparing the data and chosen model at every wavelength bin. 
    First calls the function velocity_shift, 
    
    Inputs:
    - Flux array y, flux uncertainty array yerr
    - Model spectrum y_model, obtained with get_model()
    - The BPASS SED wavelength grid rest_x, where dlambda = 1 Angstrom
    - List of spectral windows to model windowlist, which is passed into function get_wavelength_bins()
    
    Output:
    - (-1 * log(likelihood)), where log(i) means base-10 logarithm
    """  
    
    masked_spec = np.array( (y[mask] - y_model[mask]) / yerr[mask] )
    masked_err = np.array( np.sqrt(2) * np.sqrt(np.pi) * yerr[mask])
    
    resid = -0.5 * (np.dot(masked_spec, masked_spec) + np.log(np.dot(masked_err, masked_err)))

    return resid


def log_posterior(theta, x, y, yerr, modelcube, iontable, mask, add_nebular):
    """
    Calculates the (log of the) posterior probability as log(Ppos) = log(Pprior) + log(likelihood).
    
    Inputs:
    - A 4-tuple or 4-element list/array theta containing, in order, a log(age) logt, 
    a log(metallicity) logZ, an E(B-V) value ebv, and an amplitude ampl. 
    Change the limits of each parameter in the log_prior() function to fit your use case.
    - Wavelength array x, flux array y, flux uncertainty array yerr
    - Mask array. Can be generated using the get_mask function or with user-made code.
    
    Output:
    - log(posterior probability), where log(i) means base-10 logarithm
    """
    y_model = get_model(theta, x, modelcube, iontable, add_nebular)
    
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    
    ll = log_likelihood(y, yerr, y_model, mask)
    
    return lp + ll

In [ ]:
### Round to the nearest log(age) and log(metallicity) that is precomputed in the input SED suite
def nearest_age(logt):
    best_t_key, best_t = min(age_dict.items(), key=lambda x: abs(logt - x[1]))
    return best_t_key, best_t

def nearest_metallicity(logZ):
    Z = 10**logZ
    best_Z_key, best_Z = min(metal_dict.items(), key=lambda x: abs(Z - x[1]))
    return best_Z_key, best_Z

In [ ]:
def get_mask(windowlist, x):
    """
    Define one or more subsets of wavelength space to ignore when performing the data-model comparison
    (e.g., sky or geocoronal lines, ISM features).
    Inputs:
    - An Nx2 array windowlist, where each row specifies a wavelength window where weight = 0, and 
    the first and second columns respectively give the approximate lower and upper bounds of the
    window. The actual bounds will be chosen from the nearest values in the actual wavelength array.
    - Wavelength array x, defined by the spectrum being tested.
    
    Output:
    - An array of weights (1 = use in fit, 0 = do not use in fit) to be applied when evaluating log(likelihood).
    """

    mask_array = np.ones(len(x))
    
    windowmin, windowmax = np.zeros(len(windowlist)), np.zeros(len(windowlist))
    
    for k in range(len(windowlist)):
        idxmin = (np.abs(x - windowlist[k][0])).argmin()
        idxmax = (np.abs(x - windowlist[k][1])).argmin()
        windowmin[k] += x[idxmin]
        windowmax[k] += x[idxmax]
        
        mask_array[np.where( (x >= windowmin[k]) & (x <= windowmax[k]) )] = 0
    
    return np.array(mask_array, dtype='bool')

In [ ]:
def apply_red_law(x, ebv, y_model):
    """
    Redden a model spectrum for comparison with the data. 
    Inputs:
    - Wavelength array x
    - Degree of reddening ebv, which is translated to A_v under the hood
    - The model spectrum flux array y_model
    
    Output:
    - An extinguished model spectrum red_model
    """
    
    
    
    global use_red_law
    print("De-reddening spectrum assuming", use_red_law, "attenuation curve")
    
    
    if use_red_law not in ['CCM', 'Fitzpatrick99', 'ODonnell', 'FitzMassa07', 'Gordon23',
                           'Calzetti', 'SMC', 'LMC']:
        raise ValueError("'"+use_red_law+"'" +" is not a valid value for the parameter 'use_red_law'.\n \
Accepted values are 'CCM', 'Fitzpatrick99', 'ODonnell', 'FitzMassa07', 'Gordon23', Calzetti', 'LMC', and 'SMC'.")
        
    elif use_red_law == "CCM":
        red_model = extinction.apply(extinction.ccm89(x, ebv*3.1, 3.1), y_model)
        
    elif use_red_law == "Fitzpatrick99":
        red_model = extinction.apply(extinction.fitzpatrick99(x, ebv*3.1, 3.1), y_model)
        
    elif use_red_law == "ODonnell":
        red_model = extinction.apply(extinction.odonnell94(x, ebv*3.1, 3.1), y_model)
        
    elif use_red_law == "FitzMassa07":
        red_model = extinction.apply(extinction.fm07(x, ebv*3.1), y_model)
        
    elif use_red_law == "Calzetti":
        red_model = extinction.apply(extinction.calzetti00(x, ebv*4.05, 4.05), y_model)
    
    elif use_red_law == 'Gordon23':
        ext_curve = G23(Rv = 3.1)
        red_model = y_model * ext_curve.extinguish(x * u.AA, ebv * ext_curve.Rv)
        
    elif use_red_law == 'LMC':
        ext_curve = G03_LMCAvg()
        red_model = y_model * ext_curve.extinguish(x * u.AA, ebv * ext_curve.Rv)
    
    elif use_red_law == 'SMC':
        ext_curve = G03_SMCBar()
        red_model = y_model * ext_curve.extinguish(x * u.AA, ebv * ext_curve.Rv)
        
    return red_model

In [ ]:
def get_model(theta, x, modelcube, iontable, add_nebular = True):
    """
    Construct a model star cluster spectrum with values of metallicity, age, extinction, and normalization
    randomly chosen (within bounds). Choose the model nearest to the age and metallicity 
    parameters Z and t, rescale by ampl, then redden by the extinction parameter ebv.
    Default is to use the Cardelli, Clayton, & Mathis (1989) Milky Way reddening law, set with the variable
    "use_red_law". See line X for more information.
    
    Inputs:
    - A 4-tuple or 4-element list/array theta containing, in order, a log(age) logt, 
    a log(metallicity) logZ, an E(B-V) value ebv, and an amplitude ampl.

    >> Implicitly takes 2 other files/arrays as input: modelcube (defining the grid of BPASS model spectra)
    >> and iontable (to generate a nebular continuum component)
    
    Output:
    - A reddened BPASS spectrum that is nearest to the proposed values of t, Z, ebv, and ampl.
    """
    
    logt, logZ, ebv, ampl = theta
    
    met_key, met = nearest_metallicity(logZ)
    age_key, age = nearest_age(logt)
    
    nearest_stellar_model = (10**ampl) * modelcube[met_key].data[age_key]
    specwl = modelcube[met_key].data['WL'].byteswap().newbyteorder()
    
    if add_nebular == True:
        nebcont = nebular_continuum(theta, iontable)
        nearest_model = nearest_stellar_model + nebcont
    else:
        nearest_model = nearest_stellar_model
    
    red_nearest_model = apply_red_law(x, ebv, nearest_model)        
    
    return red_nearest_model

In [ ]:
"""
Python equivalent of the Starburst99 function CONTINUUM, for computing the approximate nebular continuum.
Variable 'gamma' contains emission coeffs for HI (including free-free, bound-free, and 2-photon emission)
and HeI (assuming He/H = 0.1). Values from Aller (1984) and Ferland (1980). All computations assume 
typical Case B conditions (T = 1e4 K,  f_esc = 0).

Inputs:
- A 4-tuple or 4-element list/array theta containing, in order, a log(age) logt, 
a log(metallicity) logZ, an E(B-V) value ebv, and an amplitude ampl.
- A table of ionizing photon production rates iontable

Output:
- A rough nebular continuum spectrum (in Solar luminosities per A), bounded in wavelength
by the wavelength coverage of the model cube

"""

gamma = np.array([0.,2.11e-4,5.647,9.35,9.847,10.582,16.101,24.681,26.736,
              24.883,29.979,6.519,8.773,11.545,13.585,6.333,10.444,7.023,
              9.361,7.59,9.35,8.32,9.53,8.87])*1e-40  # Units are 10**-40 ERG CM3 SEC**-1 HZ**-1
nebx = np.array([912.,913.,1300.,1500.,1800.,2200.,2855.,3331.,3421.,3422.,
             3642.,3648.,5700.,7000.,8207.,8209.,14583.,14585.,22787.,22789.,
             32813.,32815.,44680.,44682.])
alpha_B = 2.6e-13    
Qbase = 52

sparse_nebcont = (2.998e18 * gamma * 10**(Qbase)) / (alpha_B * nebx**2)


def nebular_continuum(x, theta, iontable):
    logt, logZ, _, ampl = theta
    met_key, met = nearest_metallicity(logZ)
    age_key, age = nearest_age(logt)
    
    # Interpolate the sparsely-sampled continuum to the wavelength grid of the model
    interp_function = interpolate.interp1d(nebx, sparse_nebcont, fill_value='extrapolate', )
    interp_nebcont = interp_function(x) / 3.83e33
    
    # Rescale the continuum by the emissivity of the nearest model and by the specified amplitude parameter
    Q_new = iontable[iontable['Z'] == met_key][age_key] 
    interp_nebcont = interp_nebcont * (10**(Q_new - Qbase + ampl))  
    
    return interp_nebcont

## 4) Starting a SESAMME run

Load in 1) the model cube containing the SSP suite and 2) the table of ionizing production rates you want to use in your modeling. See the supplementary script **cube_maker.py** for information about how these files are created. 

In [ ]:
# modelcube = fits.open("/path/to/file/SSP_Model_Cube.fits")
modelcube = fits.open("./synthetic_spectra/v23/afe_00/afe00_Metallicity_Table.fits")

# iontable = Table.read("/path/to/file/Model_Q_table.txt", format='ascii', header_start=0)
iontable = Table.read("./synthetic_spectra/v23/afe_00/afe00_Q_Table.txt", format='ascii', header_start=0)

Load in the spectrum of your star cluster. SESAMME assumes the spectrum has already been pre-processed (i.e., smoothed and rebinned, if necessary) and shifted to the rest frame. 

The example spectrum file contains HST/COS observations (with the G130M and G160M gratings) of a star cluster in M83 (see Hernandez et al. 2019 for details). The spectrum has been corrected for the HI absorption profile around Lyman alpha; de-reddened according to the Galactic extinction maps of Schlafly & Finkbeiner 2011; shifted to the rest frame; and smoothed and regridded to the resolution of a BPASS model suite (1 A per pix). 

In [ ]:
# Natively comes in flux density units (erg s-1 cm-2 A-1)...
specfile = Table.read("./spectra/M83/cluster8/M83-8_fhrsg.txt", format='ascii')
wl = specfile['WL']
flux = specfile['FLUX']
flux_err = specfile['ERROR']

# But rescaling to luminosity density units (L_Sun A-1) allows SESAMME to infer a stellar mass for the cluster
lum = flux * 4*np.pi * (4.8*u.Mpc.to(u.cm))**2 / 3.83e33
lum_err = flux_err * 4*np.pi * (4.8*u.Mpc.to(u.cm))**2 / 3.83e33

windowlist = np.array([[np.min(wl), 1133], [1172, 1176.5],  [1188, 1202], [1203.8, 1222], 
                       [1257, 1262], [1299, 1305], [1331, 1336.2], [1276, 1286], [1454, 1456], [1465, 1469], 
                       [1390.5, 1393.5], [1399.5, 1403.3],
                       [1523.5, 1529], [1543, 1550.], [1608.5, 1621], [1656, 1659], [1666, 1674], [1795, np.max(wl)] ])

mask = get_mask(windowlist, wl)

Set up the H5 backend, which stores the results of the MCMC process. If you want to save a second SESAMME run to the same H5 file, add a parameter *name* as illustrated below.

In [ ]:
filename = "M83_publictest.h5"
backend = emcee.backends.HDFBackend(filename, name = "Default")
backend.reset(nwalkers, ndim)


sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_posterior, backend = backend, args=(wl, lum, lum_err, 
                                          modelcube, iontable, mask, True)
)
sampler.run_mcmc(initial_pos, nsteps, progress=True);

## 5) Post-run analyses

To retrieve the results of an MCMC run, load in the relevant .H5 file. Don't forget to use the *name* parameter to specify which run to use, if multiple exist within a single .H5 file.

In [ ]:
reader = emcee.backends.HDFBackend(filename, name = 'Default')
samples = reader.get_chain()

Create a time-series plot that shows the parameter values for each walker at each step in the chain. This will also allow you to determine visually how many "burn-in steps" should be excluded from the final analysis (see below).

In [ ]:
fig, axes = plt.subplots(4, figsize=(9, 7), sharex=True)

labels = ["log(age/yr)", r"log(Z/Z$_{\odot}$)", "E(B-V)", "log(A)"]

for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

See what values are inferred for the autocorrelation time for each of the four parameters. This will determine the amount by which you should 'thin' the chain (see below). An `AutocorrError` may occur if the MCMC chain does not strictly converge, but in practice this usually can be ignored.

In [ ]:
tau = reader.get_autocorr_time()
print(tau)

Collapse the array of samples into a 4 x `nsteps` array. This is also where you:
- Discard the burn-in steps, or early iterations in the chain where the walker ensemble begins to explore the parameter space
- Thin the chain. Because the MCMC process retains some memory of previous iterations, you should only consider every *i*-th sample if you want a truly independent sampling of the posterior PDF. 

In [ ]:
# Edit the distcard and thin parameters according to the results of your run
flat_samples = reader.get_chain(discard=100, thin=100, flat=True) 
print(flat_samples.shape)

Inspect the 1-D distributions of the four parameters and their covariances. Values in `range` can be tweaked to better show the breadth of values in the posterior PDF.

You can also print the median and $\pm$1$\sigma$ values for each of the four parameters.

In [ ]:
fig = corner.corner(
    flat_samples, labels=labels, truths=[np.log10(10e6), np.log10(0.0127), None, None], quantiles = [.16, .50, .84],
    range=[(6.0,7.5), (-2.2, -1.5), (0.0,0.33), (-3, 0), ],title_kwargs={"fontsize": 14, 'weight':'semibold'}
);

In [ ]:
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))

This is an example of how to visualize the quality of fit inferred by SESAMME. Here, in the upper panel, I plot the input spectrum in black, 50 random draws from the posterior PDF in green, and the "median model" in blue (that is, the model for which each of the 4 parameters is set to the median of its 1-D distribution shown above). I also mark masked regions with grey patches.

In the lower panel, I plot the normalized residuals of those 50 random draws from the data. This shows that the MCMC process converges on a family of solutions that are good descriptions of the observed spectrum to within $\pm$ 1\% over the whole wavelength range.

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(10,6), gridspec_kw={'height_ratios': [3, 1]})


ax[0].plot(wl, lum, color='black', lw = 1, label = "Data", zorder=1)

for k in range(len(windowlist)):
    maskout = np.where((wl >= windowlist[k][0]) & (wl <= windowlist[k][1]))
    ax[0].fill_between(wl[maskout], 0, 1e6, alpha = 0.5, color='lightgrey', lw=0)
    ax[1].fill_between(wl[maskout], -10, 10, alpha = 0.5, color='lightgrey', lw=0)

## The 1DMax model
ex_star_model = (10**-1.768) * modelcube['Z010'].data['6.6']
ex_nebcont = nebular_continuum([6.5, np.log10(0.010), 0.138, -1.768], iontable)
ex_tot_model = ex_star_model + ex_nebcont
ex_tot_model = apply_red_law(x = wl, ebv = 0.109, model = ex_tot_model)

ax[0].plot(wl, ex_tot_model, color = 'royalblue', lw=1.5, label = 'Median BPASS Model', zorder=500) #'#dad786'

print("The log-probability of the 1DMax model is",
      log_posterior([6.5, np.log10(0.010), 0.109, -1.768], wl, lum, 
                    lum_err, model_cube, ion_table, mask, True))





ax[0].set_xlim(1120, 1810)
ax[1].set_xlabel(r"Wavelength ($\AA$)")
ax[0].set_ylim(0, 4.6e3)
ax[0].set_ylabel(r"L$_{\odot}$ $\AA^{-1}$")
#ax[0].set_yscale("log")
ax[0].set_title("YMC M83-8", weight='semibold')


#######

ax[1].axhline(0, color='black')

### Plot 50 random draws from the final flattened sample ensemble
i = np.random.randint(0, len(flat_samples))
t, z, ebv, amp = flat_samples[i]
nearest_t, nearest_z = nearest_age(t)[0], nearest_metallicity(z)[0]

ex_y_model = 10**(amp) * modelcube[nearest_z].data[nearest_t]
ex_nebcont = nebular_continuum([t, z, ebv, amp], iontable)
total_model = ex_y_model + ex_nebcont
total_model = apply_red_law(x = wl, ebv = ebv, model = total_model)


ax[0].plot(wl, total_model*10, alpha = 1, lw=1, color='teal', ls=":", label = "Random Draw from PDF", zorder=0)
ax[0].plot(wl, total_model, alpha = 0.1, lw=1, color='teal', ls=":", zorder=0)
ax[1].plot(wl[mask], (lum[mask] - total_model[mask])/lum[mask], alpha=  0.2, lw=1, color='royalblue', label = "Random Draw from PDF", zorder=0)

np.random.seed(99)
for i in np.random.randint(0, len(flat_samples), 49):
    t, z, ebv, amp = flat_samples[i]
    nearest_t, nearest_z = nearest_age(t)[0], nearest_metallicity(z)[0]

    ex_y_model = 10**(amp) * modelcube[nearest_z].data[nearest_t]
    ex_nebcont = nebular_continuum([t, z, ebv, amp], iontable)
    total_model = ex_y_model + ex_nebcont
    total_model = apply_red_law(x = wl, ebv = ebv, model = total_model)

    ax[0].plot(wl, total_model, alpha=  0.1, lw=1, ls=':', color='teal', zorder=0)
    ax[1].plot(wl[mask], (lum[mask] - total_model[mask])/lum[mask], ls=":", alpha=  0.05, lw=1, color='teal', zorder=0)


    
ax[0].legend(loc='best')
ax[1].set_ylabel("Residuals")
ax[1].set_ylim(-0.7,0.7)

plt.tight_layout()
# plt.savefig("fitexample.pdf", bbox_inches='tight', overwrite=True)
plt.show()